In [1]:
import keras
import numpy as np
import tensorflow as tf

from absl import flags
from pathlib import Path
from operator import itemgetter
from keras.optimizers import Adam
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score

Using TensorFlow backend.


In [2]:
import importlib

import model as nn
importlib.reload(nn)

elsa_architecture = nn.elsa_architecture

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

lang = "en"
batch_size = 250
lr = 1e-3
epochs = 100
patience = 3
data_dir = "/data/elsa2"
checkpoint_dir = "./ckpt"
optimizer = "adam"

In [ ]:
lstm_hidden = 512
lstm_drop = 0.5
final_drop = 0.5
embed_drop = 0.0
highway = False
compute_class_weight = False
multilabel = True

In [ ]:
data_dir = Path(data_dir)
wv_path = (data_dir / "{:s}_wv.npy".format(lang)).__str__()
X_path = (data_dir / "{:s}_X.npy".format(lang)).__str__()
y_path = (data_dir / "{:s}_y.npy".format(lang)).__str__()
emoji_path = (data_dir / "{:s}_emoji.txt".format(lang)).__str__()

wv = np.load(wv_path, allow_pickle=True)
input_vec = np.load(X_path, allow_pickle=True)
input_label = np.load(y_path, allow_pickle=True)

nb_tokens = len(wv)
embed_dim = wv.shape[1]
input_len = len(input_label)
nb_classes = input_label.shape[1]
maxlen = input_vec.shape[1]

train_end = int(input_len*0.7)
val_end = int(input_len*0.9)

(X_train, y_train) = (input_vec[:train_end], input_label[:train_end])
(X_val, y_val) = (input_vec[train_end:val_end], input_label[train_end:val_end])
(X_test, y_test) = (input_vec[val_end:], input_label[val_end:])

if multilabel:
    def to_multilabel(y):
        outputs = []
        for i in range(nb_classes):
            outputs.append(y[:, i])
        return outputs

    y_train = to_multilabel(y_train)
    y_val = to_multilabel(y_val)
    y_test = to_multilabel(y_test)

model = elsa_architecture(nb_classes=nb_classes,
                          nb_tokens=nb_tokens,
                          maxlen=maxlen,
                          final_dropout_rate=final_drop,
                          embed_dropout_rate=embed_drop,
                          load_embedding=True,
                          pre_embedding=wv,
                          high=highway,
                          embed_dim=embed_dim,
                          multilabel=multilabel)
model.summary()

computed_class_weight = None

if multilabel:
    loss = "binary_crossentropy"
else:
    loss = "categorical_crossentropy"
    if compute_class_weight:
        y_train_sps = []
        for row in y_train:
            y_train_sps.extend(np.where(row)[0].tolist())
        computed_class_weight = class_weight.compute_class_weight(
            'balanced', list(range(nb_classes)), y_train_sps)
        print("computed class weight = {:s}".format(str(computed_class_weight)))

if optimizer == 'adam':
    adam = Adam(clipnorm=1, lr=lr)
    model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])
elif optimizer == 'rmsprop':
    model.compile(loss=loss, optimizer='rmsprop', metrics=['accuracy'])

checkpoint_dir = Path(checkpoint_dir)
if not checkpoint_dir.exists():
    checkpoint_dir.mkdir()
checkpoint_weight_path = (checkpoint_dir / "elsa_{:s}.hdf5".format(lang)).__str__()

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto'),
    keras.callbacks.ModelCheckpoint(checkpoint_weight_path, monitor='val_loss',
                                    verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
]
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_val, y_val),
          class_weight=computed_class_weight,
          callbacks=callbacks,
          verbose=1)

freq = {line.split()[0]: int(line.split()[1]) for line in open(emoji_path).readlines()}
freq_topn = sorted(freq.items(), key=itemgetter(1), reverse=True)[:nb_classes]

if multilabel:
    y_pred = model.predict([X_test], batch_size=batch_size)
    y_pred = [np.squeeze(p) for p in y_pred]

    y_test_1d = np.array(y_test).flatten()
    y_pred_1d = np.array(y_pred).flatten()
    print(f1_score(y_test_1d, y_pred_1d > 0.5))
    print(classification_report(y_test_1d, y_pred_1d > 0.5))

    gold, pred = [], []
    for i in range(len(X_test)):
        each_gold, each_pred = [], []
        for c in range(nb_classes):
            if y_test[c][i] == 1.0:
                each_gold.append(c+1)
            else:
                each_gold.append(0)
            if y_pred[c][i] > 0.5:
                each_pred.append(c+1)
            else:
                each_pred.append(0)
        gold.extend(each_gold)
        pred.extend(each_pred)

    target_name = [""] + [e[0] for e in freq_topn]
    print(classification_report(gold, pred, target_names=target_name))
else:
    _, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    print(acc)

    y_pred = model.predict(X_test)
    print(classification_report(y_test.argmax(axis=1), y_pred.argmax(
        axis=1), target_names=[e[0] for e in freq_topn]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 200)      64132000    input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 20, 200)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectio

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 2677236 samples, validate on 764924 samples
Epoch 1/100
2677236/2677236 [==============================] - 3522s 1ms/step - loss: 5.3786 - sigmoid_0_loss: 0.4244 - sigmoid_1_loss: 0.3062 - sigmoid_2_loss: 0.2592 - sigmoid_3_loss: 0.1894 - sigmoid_4_loss: 0.1954 - sigmoid_5_loss: 0.1327 - sigmoid_6_loss: 0.1142 - sigmoid_7_loss: 0.1017 - sigmoid_8_loss: 0.0824 - sigmoid_9_loss: 0.1276 - sigmoid_10_loss: 0.0879 - sigmoid_11_loss: 0.1182 - sigmoid_12_loss: 0.0718 - sigmoid_13_loss: 0.0621 - sigmoid_14_loss: 0.1098 - sigmoid_15_loss: 0.1057 - sigmoid_16_loss: 0.0665 - sigmoid_17_loss: 0.1028 - sigmoid_18_loss: 0.0711 - sigmoid_19_loss: 0.0884 - sigmoid_20_loss: 0.0893 - sigmoid_21_loss: 0.0755 - sigmoid_22_loss: 0.0874 - sigmoid_23_loss: 0.0766 - sigmoid_24_loss: 0.0805 - sigmoid_25_loss: 0.0763 - sigmoid_26_loss: 0.0821 - sigmoid_27_loss: 0.0777 - sigmoid_

    500/2677236 [..............................] - ETA: 59:24 - loss: 5.0260 - sigmoid_0_loss: 0.4097 - sigmoid_1_loss: 0.3225 - sigmoid_2_loss: 0.2689 - sigmoid_3_loss: 0.1862 - sigmoid_4_loss: 0.2006 - sigmoid_5_loss: 0.1232 - sigmoid_6_loss: 0.0769 - sigmoid_7_loss: 0.1028 - sigmoid_8_loss: 0.0515 - sigmoid_9_loss: 0.1001 - sigmoid_10_loss: 0.0491 - sigmoid_11_loss: 0.1139 - sigmoid_12_loss: 0.0450 - sigmoid_13_loss: 0.0343 - sigmoid_14_loss: 0.0719 - sigmoid_15_loss: 0.0838 - sigmoid_16_loss: 0.0495 - sigmoid_17_loss: 0.0979 - sigmoid_18_loss: 0.0626 - sigmoid_19_loss: 0.0502 - sigmoid_20_loss: 0.0884 - sigmoid_21_loss: 0.0672 - sigmoid_22_loss: 0.0648 - sigmoid_23_loss: 0.0932 - sigmoid_24_loss: 0.0732 - sigmoid_25_loss: 0.0918 - sigmoid_26_loss: 0.0551 - sigmoid_27_loss: 0.0794 - sigmoid_28_loss: 0.0675 - sigmoid_29_loss: 0.0514 - sigmoid_30_loss: 0.0928 - sigmoid_31_loss: 0.0295 - sigmoid_32_loss: 0.0976 - sigmoid_33_loss: 0.0263 - sigmoid_34_loss: 0.0840 - sigmoid_35_loss: 0.10

2677236/2677236 [==============================] - 3190s 1ms/step - loss: 5.1060 - sigmoid_0_loss: 0.4096 - sigmoid_1_loss: 0.2959 - sigmoid_2_loss: 0.2491 - sigmoid_3_loss: 0.1837 - sigmoid_4_loss: 0.1854 - sigmoid_5_loss: 0.1267 - sigmoid_6_loss: 0.1058 - sigmoid_7_loss: 0.0933 - sigmoid_8_loss: 0.0728 - sigmoid_9_loss: 0.1226 - sigmoid_10_loss: 0.0794 - sigmoid_11_loss: 0.1139 - sigmoid_12_loss: 0.0639 - sigmoid_13_loss: 0.0568 - sigmoid_14_loss: 0.1054 - sigmoid_15_loss: 0.1013 - sigmoid_16_loss: 0.0569 - sigmoid_17_loss: 0.0965 - sigmoid_18_loss: 0.0607 - sigmoid_19_loss: 0.0818 - sigmoid_20_loss: 0.0856 - sigmoid_21_loss: 0.0704 - sigmoid_22_loss: 0.0829 - sigmoid_23_loss: 0.0736 - sigmoid_24_loss: 0.0767 - sigmoid_25_loss: 0.0720 - sigmoid_26_loss: 0.0757 - sigmoid_27_loss: 0.0750 - sigmoid_28_loss: 0.0713 - sigmoid_29_loss: 0.0560 - sigmoid_30_loss: 0.0717 - sigmoid_31_loss: 0.0637 - sigmoid_32_loss: 0.0634 - sigmoid_33_loss: 0.0304 - sigmoid_34_loss: 0.0674 - sigmoid_35_loss: 

    500/2677236 [..............................] - ETA: 1:02:40 - loss: 5.0077 - sigmoid_0_loss: 0.3845 - sigmoid_1_loss: 0.2547 - sigmoid_2_loss: 0.2649 - sigmoid_3_loss: 0.1754 - sigmoid_4_loss: 0.1745 - sigmoid_5_loss: 0.1391 - sigmoid_6_loss: 0.1135 - sigmoid_7_loss: 0.0710 - sigmoid_8_loss: 0.0665 - sigmoid_9_loss: 0.1376 - sigmoid_10_loss: 0.0560 - sigmoid_11_loss: 0.1302 - sigmoid_12_loss: 0.0566 - sigmoid_13_loss: 0.0541 - sigmoid_14_loss: 0.1057 - sigmoid_15_loss: 0.1047 - sigmoid_16_loss: 0.0592 - sigmoid_17_loss: 0.0963 - sigmoid_18_loss: 0.0627 - sigmoid_19_loss: 0.0726 - sigmoid_20_loss: 0.0778 - sigmoid_21_loss: 0.0720 - sigmoid_22_loss: 0.0650 - sigmoid_23_loss: 0.0635 - sigmoid_24_loss: 0.0964 - sigmoid_25_loss: 0.0971 - sigmoid_26_loss: 0.1330 - sigmoid_27_loss: 0.0817 - sigmoid_28_loss: 0.0695 - sigmoid_29_loss: 0.0321 - sigmoid_30_loss: 0.0819 - sigmoid_31_loss: 0.0547 - sigmoid_32_loss: 0.0783 - sigmoid_33_loss: 0.0324 - sigmoid_34_loss: 0.0640 - sigmoid_35_loss: 0.

...

2677236/2677236 [==============================] - 3064s 1ms/step - loss: 4.6866 - sigmoid_0_loss: 0.3843 - sigmoid_1_loss: 0.2786 - sigmoid_2_loss: 0.2310 - sigmoid_3_loss: 0.1713 - sigmoid_4_loss: 0.1697 - sigmoid_5_loss: 0.1186 - sigmoid_6_loss: 0.0962 - sigmoid_7_loss: 0.0828 - sigmoid_8_loss: 0.0580 - sigmoid_9_loss: 0.1114 - sigmoid_10_loss: 0.0705 - sigmoid_11_loss: 0.1061 - sigmoid_12_loss: 0.0513 - sigmoid_13_loss: 0.0497 - sigmoid_14_loss: 0.0963 - sigmoid_15_loss: 0.0931 - sigmoid_16_loss: 0.0426 - sigmoid_17_loss: 0.0868 - sigmoid_18_loss: 0.0458 - sigmoid_19_loss: 0.0723 - sigmoid_20_loss: 0.0788 - sigmoid_21_loss: 0.0631 - sigmoid_22_loss: 0.0739 - sigmoid_23_loss: 0.0685 - sigmoid_24_loss: 0.0713 - sigmoid_25_loss: 0.0655 - sigmoid_26_loss: 0.0662 - sigmoid_27_loss: 0.0679 - sigmoid_28_loss: 0.0636 - sigmoid_29_loss: 0.0486 - sigmoid_30_loss: 0.0641 - sigmoid_31_loss: 0.0574 - sigmoid_32_loss: 0.0563 - sigmoid_33_loss: 0.0265 - sigmoid_34_loss: 0.0615 - sigmoid_35_loss: 

    500/2677236 [..............................] - ETA: 56:41 - loss: 4.4570 - sigmoid_0_loss: 0.3808 - sigmoid_1_loss: 0.2871 - sigmoid_2_loss: 0.2063 - sigmoid_3_loss: 0.2072 - sigmoid_4_loss: 0.1323 - sigmoid_5_loss: 0.0817 - sigmoid_6_loss: 0.1151 - sigmoid_7_loss: 0.0367 - sigmoid_8_loss: 0.0545 - sigmoid_9_loss: 0.0969 - sigmoid_10_loss: 0.0341 - sigmoid_11_loss: 0.1093 - sigmoid_12_loss: 0.0500 - sigmoid_13_loss: 0.0334 - sigmoid_14_loss: 0.1228 - sigmoid_15_loss: 0.1018 - sigmoid_16_loss: 0.0434 - sigmoid_17_loss: 0.0711 - sigmoid_18_loss: 0.0546 - sigmoid_19_loss: 0.0392 - sigmoid_20_loss: 0.1150 - sigmoid_21_loss: 0.0492 - sigmoid_22_loss: 0.0883 - sigmoid_23_loss: 0.0668 - sigmoid_24_loss: 0.0801 - sigmoid_25_loss: 0.0670 - sigmoid_26_loss: 0.0510 - sigmoid_27_loss: 0.0516 - sigmoid_28_loss: 0.0292 - sigmoid_29_loss: 0.0533 - sigmoid_30_loss: 0.0639 - sigmoid_31_loss: 0.0682 - sigmoid_32_loss: 0.0577 - sigmoid_33_loss: 0.0114 - sigmoid_34_loss: 0.0883 - sigmoid_35_loss: 0.02

2677236/2677236 [==============================] - 3082s 1ms/step - loss: 4.6637 - sigmoid_0_loss: 0.3823 - sigmoid_1_loss: 0.2776 - sigmoid_2_loss: 0.2299 - sigmoid_3_loss: 0.1705 - sigmoid_4_loss: 0.1688 - sigmoid_5_loss: 0.1182 - sigmoid_6_loss: 0.0958 - sigmoid_7_loss: 0.0822 - sigmoid_8_loss: 0.0570 - sigmoid_9_loss: 0.1109 - sigmoid_10_loss: 0.0702 - sigmoid_11_loss: 0.1055 - sigmoid_12_loss: 0.0506 - sigmoid_13_loss: 0.0492 - sigmoid_14_loss: 0.0960 - sigmoid_15_loss: 0.0924 - sigmoid_16_loss: 0.0418 - sigmoid_17_loss: 0.0864 - sigmoid_18_loss: 0.0448 - sigmoid_19_loss: 0.0717 - sigmoid_20_loss: 0.0784 - sigmoid_21_loss: 0.0627 - sigmoid_22_loss: 0.0735 - sigmoid_23_loss: 0.0683 - sigmoid_24_loss: 0.0710 - sigmoid_25_loss: 0.0651 - sigmoid_26_loss: 0.0657 - sigmoid_27_loss: 0.0675 - sigmoid_28_loss: 0.0633 - sigmoid_29_loss: 0.0484 - sigmoid_30_loss: 0.0636 - sigmoid_31_loss: 0.0571 - sigmoid_32_loss: 0.0560 - sigmoid_33_loss: 0.0263 - sigmoid_34_loss: 0.0612 - sigmoid_35_loss: 

    500/2677236 [..............................] - ETA: 57:45 - loss: 4.3490 - sigmoid_0_loss: 0.3542 - sigmoid_1_loss: 0.2344 - sigmoid_2_loss: 0.2196 - sigmoid_3_loss: 0.1511 - sigmoid_4_loss: 0.1359 - sigmoid_5_loss: 0.1140 - sigmoid_6_loss: 0.1063 - sigmoid_7_loss: 0.0986 - sigmoid_8_loss: 0.0470 - sigmoid_9_loss: 0.1308 - sigmoid_10_loss: 0.0844 - sigmoid_11_loss: 0.1062 - sigmoid_12_loss: 0.0422 - sigmoid_13_loss: 0.0301 - sigmoid_14_loss: 0.0988 - sigmoid_15_loss: 0.0893 - sigmoid_16_loss: 0.0563 - sigmoid_17_loss: 0.0734 - sigmoid_18_loss: 0.0419 - sigmoid_19_loss: 0.0475 - sigmoid_20_loss: 0.0849 - sigmoid_21_loss: 0.0690 - sigmoid_22_loss: 0.0543 - sigmoid_23_loss: 0.0605 - sigmoid_24_loss: 0.0596 - sigmoid_25_loss: 0.0913 - sigmoid_26_loss: 0.0569 - sigmoid_27_loss: 0.0712 - sigmoid_28_loss: 0.0703 - sigmoid_29_loss: 0.0615 - sigmoid_30_loss: 0.0437 - sigmoid_31_loss: 0.0284 - sigmoid_32_loss: 0.0202 - sigmoid_33_loss: 0.0155 - sigmoid_34_loss: 0.1003 - sigmoid_35_loss: 0.05

 953500/2677236 [=========>....................] - ETA: 30:44 - loss: 4.5164 - sigmoid_0_loss: 0.3719 - sigmoid_1_loss: 0.2710 - sigmoid_2_loss: 0.2237 - sigmoid_3_loss: 0.1659 - sigmoid_4_loss: 0.1630 - sigmoid_5_loss: 0.1159 - sigmoid_6_loss: 0.0937 - sigmoid_7_loss: 0.0778 - sigmoid_8_loss: 0.0541 - sigmoid_9_loss: 0.1078 - sigmoid_10_loss: 0.0674 - sigmoid_11_loss: 0.1035 - sigmoid_12_loss: 0.0482 - sigmoid_13_loss: 0.0467 - sigmoid_14_loss: 0.0936 - sigmoid_15_loss: 0.0898 - sigmoid_16_loss: 0.0393 - sigmoid_17_loss: 0.0837 - sigmoid_18_loss: 0.0419 - sigmoid_19_loss: 0.0686 - sigmoid_20_loss: 0.0755 - sigmoid_21_loss: 0.0600 - sigmoid_22_loss: 0.0714 - sigmoid_23_loss: 0.0668 - sigmoid_24_loss: 0.0689 - sigmoid_25_loss: 0.0634 - sigmoid_26_loss: 0.0634 - sigmoid_27_loss: 0.0651 - sigmoid_28_loss: 0.0607 - sigmoid_29_loss: 0.0460 - sigmoid_30_loss: 0.0617 - sigmoid_31_loss: 0.0550 - sigmoid_32_loss: 0.0546 - sigmoid_33_loss: 0.0251 - sigmoid_34_loss: 0.0597 - sigmoid_35_loss: 0.05